In [ ]:
import pandas as pd
import gc
df_train =pd.read_csv('/home/kesci/input/bytedance/train_final.csv',names=['qId','q','aId','a','target'],nrows=100000000)
df_test =pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv',names=['qId','q','aId','a'])

In [ ]:
df_test['a']=df_test['a'].apply(lambda x:x[:-1])
df1 = df_train[['q']].copy()
df2 = df_train[['a']].copy()
df1_test = df_test[['q']].copy()
df2_test = df_test[['a']].copy()
df2.rename(columns = {'a':'q'},inplace=True)
df2_test.rename(columns = {'a':'q'},inplace=True)


In [ ]:
train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
train_questions.drop_duplicates(subset = ['q'],inplace=True)
train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.q.values).to_dict()


In [ ]:
train_cp = df_train.copy()
test_cp = df_test.copy()
train_cp.drop(['qId','aId'],axis=1,inplace=True)
test_cp['target'] = -1
test_cp.rename(columns={'aId':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])
comb['q1_hash'] = comb['q'].map(questions_dict)
comb['q2_hash'] = comb['a'].map(questions_dict)
q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()


In [ ]:
def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))


In [ ]:
train_comb = comb[comb['target'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','target']]
test_comb = comb[comb['target'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]


In [ ]:
train_comb[['q1_freq', 'q2_freq']].to_csv('train_feature_freq_final.csv', index=False)
test_comb[['q1_freq', 'q2_freq']].to_csv('test_feature_freq_final.csv', index=False)